In [16]:
import pandas as pd
from config import apiKey
import requests
import sqlalchemy as sql

In [2]:
df=pd.read_csv('source_data/california.csv')
df.head()

,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson
0,Adelanto,"34,491",276,1,20,42,213,459,136,209,114,14
1,Agoura Hills,"20,490",21,0,6,4,11,306,66,223,17,0
2,Alameda,"78,907",162,0,7,94,61,"2,579",218,"1,958",403,29
3,Albany,"20,083",40,0,8,21,11,685,105,534,46,1
4,Alhambra,"84,837",161,2,11,89,59,"1,749",259,"1,303",187,8


In [4]:
city='Adelanto, California'
url=f'https://api.geoapify.com/v1/geocode/search?text={city}&format=json&apiKey={apiKey}'

In [6]:
response=requests.get(url)
data=response.json()
data

{'results': [{'datasource': {'sourcename': 'openstreetmap',
    'attribution': '© OpenStreetMap contributors',
    'license': 'Open Database License',
    'url': 'https://www.openstreetmap.org/copyright'},
   'country': 'United States',
   'country_code': 'us',
   'state': 'California',
   'county': 'San Bernardino County',
   'city': 'Adelanto',
   'lon': -117.409215,
   'lat': 34.5827699,
   'state_code': 'CA',
   'distance': 0,
   'formatted': 'Adelanto, CA, United States of America',
   'address_line1': 'Adelanto, CA',
   'address_line2': 'United States of America',
   'category': 'administrative',
   'timezone': {'name': 'America/Los_Angeles',
    'offset_STD': '-08:00',
    'offset_STD_seconds': -28800,
    'offset_DST': '-07:00',
    'offset_DST_seconds': -25200,
    'abbreviation_STD': 'PST',
    'abbreviation_DST': 'PDT'},
   'result_type': 'city',
   'rank': {'importance': 0.6464791233868054,
    'popularity': 2.0406982932026385,
    'confidence': 1,
    'confidence_city_leve

In [8]:
lat=data['results'][0]['lat']
lon=data['results'][0]['lon']

In [12]:
for idx, row in df.head(50).iterrows(): 
    city=f'{row["City"]}, California'
    url=f'https://api.geoapify.com/v1/geocode/search?text={city}&format=json&apiKey={apiKey}'
    response=requests.get(url)
    data=response.json()
    lat=data['results'][0]['lat']
    lon=data['results'][0]['lon']
    df.loc[idx, 'lat']=lat
    df.loc[idx, 'lon']=lon

In [14]:
df.to_csv('static/data/data.csv', index=False)

In [29]:
df[df['City']=='Vernon']

,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson,lat,lon
427,Vernon,112,27,0,1,12,14,485,133,272,80,5,NaN,NaN


In [17]:
engine=sql.create_engine('sqlite:///static/data/db.sqlite')

In [20]:
df.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter', 'Rape', 'Robbery',
       'Aggravated\nassault', 'Property\ncrime', 'Burglary', 'Larceny-\ntheft',
       'Motor\nvehicle\ntheft', 'Arson', 'lat', 'lon'],
      dtype='object')

In [19]:
df.to_sql('crime', con=engine, index=False, if_exists='replace')

In [22]:
results=engine.execute('select * from crime').all()
type(results)

list

In [25]:
data=[]
for each_result in results: 
    data.append(list(each_result))

In [28]:
data[-1]

['Yucca Valley',
 '21,858',
 '160',
 0,
 '9',
 '25',
 '126',
 '292',
 '84',
 '157',
 '51',
 '3',
 None,
 None]